In [7]:
pip install py-zabbix

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from pyzabbix import ZabbixAPI

# Create ZabbixAPI class instance
zapi = ZabbixAPI(url='http://10.10.10.20/zabbix', user='Admin', password='zabbix')

# Get all disabled hosts
result = zapi.host.get(status=1)
hostnames = [host['host'] for host in result1]

# Other way to get all disabled hosts
result2 = zapi.do_request('host.get', {'status':1})
hostnames2 = [host['host'] for host in result2['result']]

TypeError: __init__() got an unexpected keyword argument 'url'